In [1]:
from app_store_scraper import AppStore

In [2]:
import pandas as pd
!pip install numpy==1.24.3
import numpy as np

from app_store_web_scraper import AppStoreEntry

data = AppStore(country='us', app_name='Coinbase: Buy BTC, ETH, SOL', app_id = '886427730')
data.review(how_many=1000)
data.reviews

  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 137, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "/opt/anaconda3

2025-04-24 15:46:04,885 [INFO] Base - Initialised: AppStore('us', 'coinbase-buy-btc-eth-sol', 886427730)
2025-04-24 15:46:04,886 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/coinbase-buy-btc-eth-sol/id886427730
2025-04-24 15:46:04,993 [ERROR] Base - Something went wrong: Expecting value: line 1 column 1 (char 0)
2025-04-24 15:46:04,993 [INFO] Base - [id:886427730] Fetched 0 reviews (0 fetched in total)


[]

In [3]:
coinbase = pd.DataFrame(np.array(data.reviews),columns=['review'])
coinbase_df= coinbase.join(pd.DataFrame(coinbase.pop('review').tolist()))
coinbase_df.head()

""


In [4]:
# save_filepath = '/Users/graceliu/Desktop/Columbia/Spring 2025/NLP/final_project/coinbase_reviews.csv'
# coinbase_df.to_csv(save_filepath, index=False)

# Preprocessing

In [5]:
import re

def clean_text(str_in):
    str_in = str_in.replace("’", "'")
    data_clean_t = re.sub("[^A-Za-z']+", " ", str_in).strip().lower()
    return data_clean_t

coinbase_df['review'] = coinbase_df['review'].apply(clean_text)
coinbase_df.head(10)
# save_clean_filepath = '/Users/graceliu/Desktop/Columbia/Spring 2025/NLP/final_project/coinbase_reviews_cleaned.csv'
# coinbase_df.to_csv(save_clean_filepath, index = False)

KeyError: 'review'

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def rem_sw(str_in):
    import nltk
    from nltk.corpus import stopwords
    sw = stopwords.words('english')
    token_fun = str_in.split()
    ar = list()
    for word in token_fun:
        if word not in sw:
            ar.append(word)
    ar = " ".join(ar)
    return ar

coinbase_df['review'] = coinbase_df['review'].apply(rem_sw)
save_clean_filepath = '/Users/graceliu/Desktop/Columbia/Spring 2025/NLP/final_project/coinbase_reviews_cleaned.csv'
coinbase_df.to_csv(save_clean_filepath, index = False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/graceliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
coinbase_df['review'] = coinbase_df['review'].astype(str)
coinbase_df.head()

,date,developerResponse,review,rating,isEdited,userName,title,ml_rating
0,2021-10-11 01:28:16,"{'id': 25674719, 'body': 'Hi there, thank you ...",impressed easy app use endless amounts informa...,5,False,dank0116,Easy money !,5
1,2025-01-23 03:06:29,"{'id': 49919909, 'body': 'Hi there, Stevenra14...",really good app crypto beginners intermediate ...,4,False,Stevenra14,"Great app, but missing a few key things…",4
2,2024-10-22 02:53:25,"{'id': 47669014, 'body': 'Hello Robinhoodallda...",use coinbase three years happy service halfway...,1,False,Robinhoodallday,Used for years but has gone to crap,1
3,2024-11-11 20:08:17,"{'id': 48168788, 'body': 'Hi CPG Tiberius, we'...",absolutely mind boggling unhelpful circular cu...,1,False,CPG Tiberius,Worst Customer Support,1
4,2025-04-12 14:35:28,"{'id': 51660061, 'body': 'Hi Usjnendin, we’re ...",left coinbase years ago kept removing payment ...,1,True,Usjnendin,Holding Money Hostage,1


# Most frequent words and most frequent rare words visualizations

In [ ]:
import altair as alt
from collections import Counter

# Combine all reviews into a single string 
all_reviews_words = " ".join(coinbase_df['review'])
all_words = all_reviews_words.split()

# Get top 10 words
top_words = pd.DataFrame(Counter(all_words).most_common(10), columns=["word", "word_count"])

chart = (
    alt.Chart(top_words)
    .mark_bar(color="blue")
    .encode(
        x=alt.X("word_count:Q", title="Count"),
        y=alt.Y("word:N", sort="-x", title=""),
    )
)

# Word labels inside bars
text_word = chart.mark_text(
    align="left",
    dx=5,  # Adjust spacing inside the bar
    color="white",
    fontSize=14
).encode(text="word:N")

# Count labels to the right of bars
text_count = chart.mark_text(
    align="right",
    dx=-5,  # Adjust spacing
    color="white",
    fontSize=14
).encode(text="word_count:Q")

# Combine and adjust title
final_chart = (
    (chart + text_word + text_count)
    .properties(
        title=alt.TitleParams(
            "Most Frequent Terms Coinbase Reviews",
            fontSize=20,  # Larger title
            anchor="middle",
        )
    )
)
final_chart

alt.LayerChart(...)

### ^ not very helpful, can do a similar visualization to this after tf-idf 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_tfidf(df, text_col, min_n=1, max_n=1):
    vectorizer = TfidfVectorizer(ngram_range=(min_n, max_n), stop_words = 'english')

    # Fit and transform the review text
    tfidf_matrix = vectorizer.fit_transform(df[text_col])

    # Create DataFrame with feature names
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return tfidf_df

tfidf_reviews = compute_tfidf(coinbase_df, text_col='review', min_n=1, max_n=1)
tfidf_reviews.head()


,abandon,abandoned,abide,ability,able,abra,abroad,abruptly,absent,absolute,...,youve,yrs,yubikey,yup,zero,zeros,zfv,zodiac,zoom,zooming
0,0.0,0.0,0.0,0.000000,0.065094,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.078138,0.000000,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.079917,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.09234,...,0.0,0.0,0.0,0.0,0.070013,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.054767,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
avg_tfidf = tfidf_reviews.mean().sort_values(ascending=False).head(10)
top_tfidf_df = avg_tfidf.reset_index()
top_tfidf_df.columns = ['word', 'avg_tfidf']

chart = (
    alt.Chart(top_tfidf_df)
    .mark_bar(color="blue")
    .encode(
        x=alt.X("avg_tfidf:Q", title="Average TF-IDF Score"),
        y=alt.Y("word:N", sort="-x", title=""),
    )
)

# Word labels inside bars
text_word = chart.mark_text(
    align="left",
    dx=5,
    color="white",
    fontSize=14
).encode(text="word:N")

# Score labels to the right of bars
text_score = chart.mark_text(
    align="right",
    dx=-5,
    color="white",
    fontSize=14
).encode(text="avg_tfidf:Q")

# Combine layers and title
final_chart = (
    (chart + text_word + text_score)
    .properties(
        title=alt.TitleParams(
            "Top 10 Terms by TF-IDF Score in Coinbase Reviews",
            fontSize=20,
            anchor="middle",
        )
    )
)

final_chart

alt.LayerChart(...)

### ^still not very helpful

# Sentiment Analysis (vader)
### good for short texts and detecting polarity

In [ ]:
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
coinbase_df['vader_scores'] = coinbase_df['review'].apply(analyzer.polarity_scores)
coinbase_df[['vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = coinbase_df['vader_scores'].apply(pd.Series)

In [ ]:
def label_vader_sentiment(compound_score):
    if compound_score > 0.963:
        return "positive"
    else:
        return "negative"
    # elif compound_score < 0.8:
    #     return "negative"
    # else:
    #     return "neutral"

coinbase_df['vader_sentiment'] = coinbase_df['vader_compound'].apply(label_vader_sentiment)
coinbase_df.head()

,date,developerResponse,review,rating,isEdited,userName,title,ml_rating,vader_scores,vader_neg,vader_neu,vader_pos,vader_compound,vader_sentiment
0,2021-10-11 01:28:16,"{'id': 25674719, 'body': 'Hi there, thank you ...",impressed easy app use endless amounts informa...,5,False,dank0116,Easy money !,5,"{'neg': 0.063, 'neu': 0.662, 'pos': 0.275, 'co...",0.063,0.662,0.275,0.9717,positive
1,2025-01-23 03:06:29,"{'id': 49919909, 'body': 'Hi there, Stevenra14...",really good app crypto beginners intermediate ...,4,False,Stevenra14,"Great app, but missing a few key things…",4,"{'neg': 0.059, 'neu': 0.635, 'pos': 0.306, 'co...",0.059,0.635,0.306,0.9888,positive
2,2024-10-22 02:53:25,"{'id': 47669014, 'body': 'Hello Robinhoodallda...",use coinbase three years happy service halfway...,1,False,Robinhoodallday,Used for years but has gone to crap,1,"{'neg': 0.057, 'neu': 0.764, 'pos': 0.179, 'co...",0.057,0.764,0.179,0.9517,negative
3,2024-11-11 20:08:17,"{'id': 48168788, 'body': 'Hi CPG Tiberius, we'...",absolutely mind boggling unhelpful circular cu...,1,False,CPG Tiberius,Worst Customer Support,1,"{'neg': 0.074, 'neu': 0.736, 'pos': 0.19, 'com...",0.074,0.736,0.190,0.8542,negative
4,2025-04-12 14:35:28,"{'id': 51660061, 'body': 'Hi Usjnendin, we’re ...",left coinbase years ago kept removing payment ...,1,True,Usjnendin,Holding Money Hostage,1,"{'neg': 0.026, 'neu': 0.767, 'pos': 0.207, 'co...",0.026,0.767,0.207,0.9778,positive


In [ ]:
coinbase_df['vader_sentiment'].value_counts()

vader_sentiment
negative    866
positive    134
Name: count, dtype: int64

## tf-idf in negative reviews

In [ ]:
neg_reviews = coinbase_df[coinbase_df['vader_sentiment'] == 'negative'].copy()
tfidf_neg = compute_tfidf(neg_reviews, 'review')

neg_rare_words = tfidf_neg.mean().sort_values().head(15)
neg_rare_words.columns = ['word', 'avg_tfidf']
print(neg_rare_words)

parameters     0.000101
permanent      0.000101
dialogue       0.000101
south          0.000113
machine        0.000113
replay         0.000113
border         0.000113
ca             0.000113
korean         0.000113
acted          0.000113
shift          0.000113
neglect        0.000119
tender         0.000119
excessively    0.000119
convenience    0.000119
dtype: float64


### doesn't tell us much

In [ ]:
# recurring themes (most common words in neg reviews)
all_words = " ".join(neg_reviews['review']).split()
word_counts = Counter(all_words)
common_neg_words = pd.DataFrame(word_counts.most_common(20), columns=['word', 'count'])
common_neg_words

,word,count
0,coinbase,1294
1,account,1085
2,app,975
3,money,871
4,crypto,590
5,get,542
6,support,423
7,customer,409
8,bank,390
9,even,386


### customer support/service, fees are pain points. "time" so also maybe how long the app takes to transfer funds, etc.

# Topic Modeling (LDA)

## LDA using gensim (USE THIS, not sklearn)

In [ ]:
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
# 1. Tokenize and preprocess
texts = [simple_preprocess(doc, deacc=True) for doc in neg_reviews['review']]

# 2. Create dictionary and bag-of-words corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 3. Train LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=4,
                     random_state=42,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

# 4. Print top words per topic
for i, topic in lda_model.show_topics(num_topics=4, formatted=False):
    print(f"Topic {i+1}: {[word for word, _ in topic]}")

2025-04-18 17:52:29,086 [INFO] gensim.corpora.dictionary - adding document #0 to Dictionary<0 unique tokens: []>
2025-04-18 17:52:29,107 [INFO] gensim.corpora.dictionary - built Dictionary<5716 unique tokens: ['account', 'anything', 'ask', 'aware', 'back']...> from 866 documents (total 63027 corpus positions)
2025-04-18 17:52:29,108 [INFO] gensim.utils - Dictionary lifecycle event {'msg': "built Dictionary<5716 unique tokens: ['account', 'anything', 'ask', 'aware', 'back']...> from 866 documents (total 63027 corpus positions)", 'datetime': '2025-04-18T17:52:29.108214', 'gensim': '4.3.3', 'python': '3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 08:22:19) [Clang 14.0.6 ]', 'platform': 'macOS-14.3-arm64-arm-64bit', 'event': 'created'}
2025-04-18 17:52:29,123 [INFO] gensim.models.ldamodel - using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2025-04-18 17:52:29,123 [INFO] gensim.models.ldamodel - using symmetric eta at 0.25
2025-04-18 17:52:29,124 [INFO] gensim.models

Topic 1: ['money', 'account', 'coinbase', 'bank', 'app', 'fees', 'back', 'get', 'buy', 'bitcoin']
Topic 2: ['app', 'crypto', 'coinbase', 'like', 'even', 'see', 'one', 'coin', 'use', 'time']
Topic 3: ['coinbase', 'app', 'crypto', 'money', 'buy', 'time', 'get', 'fees', 'would', 'use']
Topic 4: ['account', 'coinbase', 'money', 'customer', 'support', 'app', 'get', 'service', 'email', 'crypto']


### Insights
1. customer support (also phone, email, number, access) --> complaints about not reaching customer support --> improve customer support and communication
2. account, money --> frustration around accessing account/money --> streamline account recovery process
3. fees --> unclear or high fees --> improve fee transparency
4. time --> bad timing of transactions or delays --> improve reliabliity and speed of banking/crypto transfers

## LDA using sklearn

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(neg_reviews['review'])

lda = LatentDirichletAllocation(n_components=4, random_state=42)
lda.fit(X)

# Top words per topic
for i, topic in enumerate(lda.components_):
    print(f"Topic {i+1}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Topic 1:
['fees', 'time', 'customer', 'bank', 'support', 'crypto', 'money', 'app', 'coinbase', 'account']
Topic 2:
['jacob', 'texted', 'phantom', 'armstrong', 'blocks', 'brian', 'excites', 'temp', 'sales', 'felt']
Topic 3:
['bnt', 'adjust', 'homepage', 'quit', 'men', 'jeremy', 'coordinated', 'renner', 'battery', 'bonus']
Topic 4:
['tolerated', 'angela', 'wi', 'bless', 'brian', 'fungible', 'passcode', 'wannabe', 'sendable', 'barcodes']


### Insights from topics 1 & 2
1. customer support (also phone, email, number, access) --> complaints about not reaching customer support --> improve customer support and communication
2. account, money --> frustration around accessing account/money --> streamline account recovery process
3. fees --> unclear or high fees --> improve fee transparency
4. time --> bad timing of transactions or delays --> improve reliabliity and speed of banking/crypto transfers

### Insights from topic 3: emotional cluster, likely angry reviews 
1. unimpressive, interfaces --> poor UI/UX --> simplify in-app navigation or run user/usability studies focusing on UI

### nothing useful from topic 4

# Sentiment Analysis (textblob)
### adds subjectivity, better for longer reviews

In [ ]:
# !pip install textblob 
# nltk.download('punkt')

In [ ]:
# from textblob import TextBlob
# '''
# polarity: [-1.0, 1.0]:
# - Negative = negative sentiment
# - Positive = positive sentiment
# - 0 = neutral
# subjectivity: 0.0, 1.0]:
# - Closer to 1 = opinion-based
# - Closer to 0 = more factual/objective
# '''
# def add_textblob_sentiment(df, text_col):
#     df[text_col] = df[text_col].astype(str)

#     # Apply TextBlob sentiment
#     df['textblob_polarity'] = df[text_col].apply(lambda x: TextBlob(x).sentiment.polarity)
#     df['textblob_subjectivity'] = df[text_col].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
#     return df

# coinbase_df_with_tb = add_textblob_sentiment(coinbase_df, 'review')
# coinbase_df_with_tb[['review', 'textblob_polarity', 'textblob_subjectivity']].head(10)

## not sure how to feel about text blob's polarity scores after cross referencing them with the original coinbase reviews and ratings